In [14]:
from operator import itemgetter
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.messages import HumanMessage, AIMessage,SystemMessage, trim_messages
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


In [15]:
# 👇 Use your Windows IP here
llm = Ollama(
    model="llama3",
    base_url="http://192.168.1.3:11434"  # <- ← 🧠 IP address where ollama is running, given that we are using WSL.
)

In [16]:
trimmer = trim_messages(
    max_tokens=1024,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=True,
    start_on="human"
)

In [17]:
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """Get the chat history for a given session ID."""
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]


In [36]:
system_prompt = (
    "You are a knowledgeable assistant. "
    "Answer user questions clearly and concisely. "
    "Respond only with the answer unless clarification is needed.")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages")
     ]
)

In [ ]:
chain=(RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | llm)

In [38]:
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi, i am Kaleem"),
    AIMessage(content="Hi Salim, how can I assist you today?"),
    HumanMessage(content="I am a data scientist and i want to learn about MLops"),
    AIMessage(content="That's great! MLops is a crucial aspect of deploying and maintaining machine learning models in production. It involves practices and tools that help automate and streamline the process of model deployment, monitoring, and management."),
]

In [39]:
# response = chain.invoke({
#         "messages": messages + [HumanMessage(content="do u remember my name")]
#     }
# )
# response

In [43]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages",history_messages_key="messages")
config = {"configurable": {"session_id": "chat3"}}
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="capital of France?")]
        
    }, config=config)
response

"I'm ready to help! What's your question?"